In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



C:\Users\Sai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


100%|██████████| 170498071/170498071 [00:06<00:00, 26509767.99it/s]


Extracting ./CIFAR-10\cifar-10-python.tar.gz to ./CIFAR-10
Files already downloaded and verified


C:\Users\Sai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Sai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/10], Loss: 1.1183
Epoch [2/10], Loss: 0.6681
Epoch [3/10], Loss: 1.1032
Epoch [4/10], Loss: 1.0594
Epoch [5/10], Loss: 0.1973
Epoch [6/10], Loss: 0.3466
Epoch [7/10], Loss: 0.3964
Epoch [8/10], Loss: 0.4449
Epoch [9/10], Loss: 0.2159
Epoch [10/10], Loss: 0.2009
Test Accuracy: 76.75%


: 

In [ ]:
# Hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 15

In [ ]:
# CIFAR-10 data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download CIFAR-10 dataset and apply transformations
train_dataset = torchvision.datasets.CIFAR10(root='./CIFAR-10', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./CIFAR-10', train=False, download=True, transform=transform)

# Create DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Import ResNet-18 model from torchvision
resnet18_model = torchvision.models.resnet18(pretrained=False, num_classes=10)  # Set num_classes to 10 for CIFAR-10
resnet18_model.to(device)


In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18_model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Training loop
for epoch in range(num_epochs):
    resnet18_model.train()  # Set the model to training mode
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = resnet18_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Print training statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



In [ ]:
# Evaluation on the test set
resnet18_model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = resnet18_model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')